#### Process ATC data into a multi-label format

In [1]:
import pandas as pd
import numpy as np

In [2]:
atc_drug_df = pd.read_csv('/home/liujin/Offtarget_drugsafety/offtarget_application/ATC_classification/data/atc_drug_label.csv')
atc_drug_df.head(2)

,smiles,label
0,CC1=NC=C(COP(=O)(O)O)C(C=O)=C1O,Alimentary tract and metabolism
1,CN(C)C1=CC=C2N=C3C=CC(=[N+](C)C)C=C3SC2=C1,Various


In [3]:
from collections import defaultdict
atc_drug_smiles_dict = defaultdict(list)
for i in range(len(atc_drug_df)):
    try: 
        for j in atc_drug_df.iloc[i]['label'].split(','):
            j = str(j).strip()
            atc_drug_smiles_dict[j].append(atc_drug_df.iloc[i]['smiles'])
    except: 
        atc_drug_smiles_dict[atc_drug_df.iloc[i]['label']].append(atc_drug_df.iloc[i]['smiles'])

print(len(atc_drug_smiles_dict)) # 14 ATC codes
print(atc_drug_smiles_dict.keys())

14
dict_keys(['Alimentary tract and metabolism', 'Various', 'Nervous system', 'Systemic hormonal preparations', 'Antiinfectives for systemic use', 'Blood and blood-forming organs', 'Sensory organs', 'Cardiovascular system', 'Dermatologicals', 'Respiratory system', 'Genito urinary system and sex hormones', 'Antiparasitic products', 'Musculo-skeletal system', 'Antineoplastic and immunomodulating agents'])


In [4]:
atc_drug_smiles_dict['A'].append(atc_drug_smiles_dict.pop('Alimentary tract and metabolism'))
atc_drug_smiles_dict['V'].append(atc_drug_smiles_dict.pop('Various'))
atc_drug_smiles_dict['N'].append(atc_drug_smiles_dict.pop('Nervous system'))
atc_drug_smiles_dict['H'].append(atc_drug_smiles_dict.pop('Systemic hormonal preparations'))
atc_drug_smiles_dict['J'].append(atc_drug_smiles_dict.pop('Antiinfectives for systemic use'))
atc_drug_smiles_dict['B'].append(atc_drug_smiles_dict.pop('Blood and blood-forming organs'))
atc_drug_smiles_dict['S'].append(atc_drug_smiles_dict.pop('Sensory organs'))
atc_drug_smiles_dict['C'].append(atc_drug_smiles_dict.pop('Cardiovascular system'))
atc_drug_smiles_dict['D'].append(atc_drug_smiles_dict.pop('Dermatologicals'))
atc_drug_smiles_dict['R'].append(atc_drug_smiles_dict.pop('Respiratory system'))
atc_drug_smiles_dict['G'].append(atc_drug_smiles_dict.pop('Genito urinary system and sex hormones'))
atc_drug_smiles_dict['P'].append(atc_drug_smiles_dict.pop('Antiparasitic products'))
atc_drug_smiles_dict['M'].append(atc_drug_smiles_dict.pop('Musculo-skeletal system'))
atc_drug_smiles_dict['L'].append(atc_drug_smiles_dict.pop('Antineoplastic and immunomodulating agents'))

In [18]:
print(len(atc_drug_smiles_dict))
print(atc_drug_smiles_dict.keys())
print(atc_drug_smiles_dict)
print(len(atc_drug_smiles_dict['A'][0]))
print(atc_drug_smiles_dict['A'][0])

14
dict_keys(['A', 'V', 'N', 'H', 'J', 'B', 'S', 'C', 'D', 'R', 'G', 'P', 'M', 'L'])
defaultdict(<class 'list'>, {'A': [['CC1=NC=C(COP(=O)(O)O)C(C=O)=C1O', 'C=C1CCC(O)CC1=CC=C1CCCC2(C)C1CCC2C(C)CCCC(C)(C)O', 'CC(C)[N+](C)(CCC(C(N)=O)(C1=CC=CC=C1)C1=CC=CC=C1)C(C)C', 'CC12C=CC(=O)C=C1CCC1C2C(=O)CC2(C)C1CCC2(O)C(=O)CO', 'CC(C)[N+](C)(CCOC(=O)C1C2=CC=CC=C2OC2=CC=CC=C21)C(C)C', 'C[N+]1(C)CCN(CC(O)(C2=CC=CC=C2)C2CCCCC2)CC1', 'C[N+]1(C)CCCC(OC(=O)C(O)(C2=CC=CC=C2)C2=CC=CC=C2)C1', 'CC[N+](C)(CC)CCOC(=O)C1C2=CC=CC=C2OC2=CC=CC=C21', 'CC[N+](CC)(CC)CCC(O)(C1=CC=CC=C1)C1CCCCC1', 'COC1CC(=C(C2=CC=CS2)C2=CC=CS2)C[N+](C)(C)C1', 'O=S(=O)([O-])OC1=CC=C(C(C2=CC=C(OS(=O)(=O)[O-])C=C2)C2=CC=CC=N2)C=C1', 'NC(=O)C1=C[N+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(N4C=NC5=C(N)N=CN=C54)C(O)C3O)C(O)C2O)=CC=C1', 'OO', 'NC(=O)CCC(N)C(=O)O', 'O=C1OC(C(O)CO)C(O)=C1O', 'OCC(O)CO', 'O=C(O)CCCCC1SCC2NC(=O)NC21', 'NC(=O)C1=CN=CC=C1', 'O=C(O)CC(O)(CC(=O)O)C(=O)O', 'NC(=O)CCCCC1CCSS1', 'CC1=CC2=C(C=C1C)N(CC(O)C(O)C(O)CO)C1=NC(=O)NC

In [19]:
# Transform the dictionary into a DataFrame, where columns represent 14 different labels, and rows correspond to SMILES (Simplified Molecular Input Line Entry System) strings. Assign a label value of 1 for each SMILES string that has a relationship with a label, and a label value of 0 for those without any association

atc_drug_smiles_df = pd.DataFrame(columns=['smiles','A', 'V', 'N', 'H', 'J', 'B', 'S', 'C', 'D', 'R', 'G', 'P', 'M', 'L'])

smiles_list = atc_drug_df['smiles'].tolist() #3491个smiles

for smiles in smiles_list:
    smiles_label_list = []
    smiles_label_list.append(smiles)
    for label in atc_drug_smiles_dict.keys():
        if smiles in atc_drug_smiles_dict[label][0]:
            smiles_label_list.append(1)
        else:
            smiles_label_list.append(0)
    atc_drug_smiles_df.loc[len(atc_drug_smiles_df)] = smiles_label_list


In [20]:
atc_drug_smiles_df.head(5)

,smiles,A,V,N,H,J,B,S,C,D,R,G,P,M,L
0,CC1=NC=C(COP(=O)(O)O)C(C=O)=C1O,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,CN(C)C1=CC=C2N=C3C=CC(=[N+](C)C)C=C3SC2=C1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,O=C(O)CCCO,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,NC(CC1=CC(I)=C(O)C(I)=C1)C(=O)O,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,C=C1CCC(O)CC1=CC=C1CCCC2(C)C1CCC2C(C)CCCC(C)(C)O,1,0,0,1,0,0,0,0,0,0,0,0,0,0


In [21]:
number_1 = 0
number_0 = 0
for i in range(1,15):
    number_1 += atc_drug_smiles_df.iloc[:,i].value_counts()[1]
    number_0 += atc_drug_smiles_df.iloc[:,i].value_counts()[0]
print(number_1)
print(number_0)

4253
44621


In [22]:
atc_drug_smiles_df.to_csv('/home/liujin/Offtarget_drugsafety/offtarget_application/ATC_classification/data/atc_drug_smiles_label.csv', index=False)

#### Dataset segmentation

In [23]:
from random import random
import numpy as np
import pandas as pd
import sys
import os
from os.path import join as path_join

from skmultilearn.model_selection import iterative_train_test_split

In [28]:
def load_raw_data(path,label=None):
    '''
    Custom data loading function specific for my preprocessed dataset, 
    depends on format of saved data
    '''
    data = pd.read_csv(path) 
    X = data.smiles
    y = data.iloc[:,1:] 
    
    if label:
        y = y.loc[:,label]
    
    return X, y

def split_data(X, y, test_split = 0.2,save_path = None):
    ''' 
    Function to split the data into train/test datasets
    '''
    if isinstance(X, pd.DataFrame): # if dataframe
        X_column_names = X.columns
        X = np.matrix(X)  
    elif isinstance(X, pd.Series): # if series
        X_column_names = [X.name]
        X = np.matrix(X.values).T
    elif isinstance(X, np.matrix): # if matrix
        X_column_names = None
    elif not isinstance(X, np.matrix): # if neither
        raise Exception(f'X must be an object of type np.matrix or pd.DataFrame and not {type(X)}')
        
    # handle y input and extract column names if possible
    if isinstance(y, pd.DataFrame): # if dataframe
        y_column_names = y.columns
        y = np.matrix(y) 
    elif isinstance(y, pd.Series): # if series
        y_column_names = [y.name]
        y = np.matrix(y.values).T
    elif isinstance(X, np.matrix): # if matrix
        y_column_names = None
    elif not isinstance(y, np.matrix): # if neither
        raise Exception(f'y must be an object of type np.matrix or pd.DataFrame and not {type(y)}')
        
    # make sure X and y are the same size
    assert y.shape[0] == X.shape[0], f'X and y do not have the same number of samples, {X.shape[0]} and {y.shape[0]}'
    
    X_train, y_train, X_test, y_test = iterative_train_test_split(X,y,test_size = test_split) 
    
    
    # convert back to dataframe
    X_train = pd.DataFrame(X_train,columns=X_column_names)
    y_train = pd.DataFrame(y_train,columns=y_column_names)
    X_test = pd.DataFrame(X_test,columns=X_column_names)
    y_test = pd.DataFrame(y_test,columns=y_column_names)
    
    # if save path is provided then save the split data separately
    if save_path:
    
        # save train data
        pd.concat([X_train,y_train],axis=1).to_csv(path_join(save_path,'train.csv'),index=False)
        # save test data
        pd.concat([X_test,y_test],axis=1).to_csv(path_join(save_path,'test.csv'),index=False)
     
    # just return the split data and don't save
    else:
        return X_train, y_train, X_test, y_test

In [29]:
X,y = load_raw_data('/home/liujin/Offtarget_drugsafety/offtarget_application/ATC_classification/data/atc_drug_smiles_label.csv')

split_data(X,y,save_path='/home/liujin/Offtarget_drugsafety/offtarget_application/ATC_classification/data')